In [1]:
import sys, os; sys.path.append(os.path.dirname(os.getcwd())) 

import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from matplotlib.cm import get_cmap

# from ipywidgets import *

from typing import Union

import numpy as np
import pandas as pd
import pickle
from datetime import datetime, timedelta
from tqdm import tqdm
from scipy.linalg import expm
from sklearn.model_selection import train_test_split
from sklearn.utils.validation import check_is_fitted
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import wishart
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import GridSearchCV
# from sklearn import neighbors, clone


from pyfrechet.metric_spaces import MetricData, Euclidean, LogCholesky, spd_to_log_chol, log_chol_to_spd
# from pyfrechet.regression.frechet_regression import LocalFrechet, GlobalFrechet
# from pyfrechet.regression.kernels import NadarayaWatson, gaussian, epanechnikov
# from pyfrechet.regression.knn import KNearestNeighbours
from pyfrechet.regression.bagged_regressor import BaggedRegressor
from pyfrechet.regression.trees import Tree
from pyfrechet.metrics import mse

INFO: Using numpy backend


# Data preparation

In [2]:
weather_data_jan=pd.read_excel('NY Weather Data.xlsx', sheet_name='January 2019 NY', skiprows=1)
weather_data_feb=pd.read_excel('NY Weather Data.xlsx', sheet_name='February 2019 NY', skiprows=1)

weather_colnames=['Day', 'Temp.Max', 'Temp.Avg', 'Temp.Min', 'DewPoint.Max', 'DewPoint.Avg', 'DewPoint.Min', 
                  'Humidity.Max', 'Humidity.Avg', 'Humidity.Min', 'WindSpeed.Max', 'WindSpeed.Avg', 'WindSpeed.Min',
                  'Pressure.Max', 'Pressure.Avg', 'Pressure.Min', 'Precipitation']

weather_data_jan.columns=weather_colnames
weather_data_feb.columns=weather_colnames

print(weather_data_jan.shape)
print(weather_data_feb.shape)
print(weather_data_jan.isna().sum().sum())
print(weather_data_feb.isna().sum().sum())

(31, 17)
(28, 17)
0
0


In [17]:
weather_data_jan.head()

,Day,Temp.Max,Temp.Avg,Temp.Min,DewPoint.Max,DewPoint.Avg,DewPoint.Min,Humidity.Max,Humidity.Avg,Humidity.Min,WindSpeed.Max,WindSpeed.Avg,WindSpeed.Min,Pressure.Max,Pressure.Avg,Pressure.Min,Precipitation
0,1,16,10.6,5,12,5.6,-3,97,73.5,50,44,24.7,0,1021.0,1008.3,1001.3,35.31
1,2,5,3.0,2,-2,-5.3,-6,62,54.9,50,24,13.7,6,1027.4,1023.8,1018.2,0.00
2,3,7,5.5,4,3,-0.9,-6,86,64.7,45,33,16.7,0,1017.2,1013.4,1010.8,0.00
3,4,8,5.8,3,3,-1.7,-5,71,59.1,50,28,15.8,6,1015.9,1012.9,1007.1,0.00
4,5,8,6.7,6,6,4.7,1,96,88.2,68,33,21.9,9,1006.1,1000.1,994.5,5.84


In [4]:
taxi_zones=pd.read_csv('taxi+_zone_lookup.csv')
# list(taxi_zones[taxi_zones.Borough=='Manhattan'].Zone)
taxi_zones[taxi_zones.Borough=='Manhattan'][['Zone']]

,Zone
3,Alphabet City
11,Battery Park
12,Battery Park City
23,Bloomingdale
40,Central Harlem
...,...
245,West Chelsea/Hudson Yards
248,West Village
260,World Trade Center
261,Yorkville East


In [5]:
# Aggregate the different zones of Manhattan in the 10 groups
zones_correspondence={
    1: ['Central Harlem', 'Central Harlem North', 'Central Park', 'East Harlem North', 'East Harlem South',
        'Hamilton Heights', 'Inwood', 'Inwood Hill Park', 'Washington Heights North', 'Washington Heights South'],
    2: ['Bloomingdale', 'Manhattan Valley', 'Manhattanville', 'Morningside Heights', 'Upper West Side North', 
        'Upper West Side South'],
    3: ['Lenox Hill East', 'Lenox Hill West', 'Upper East Side North', 'Upper East Side South', 'Yorkville East',
        'Yorkville West'],
    4: ['Clinton East', 'Clinton West', 'East Chelsea', 'Lincoln Square East', 'Lincoln Square West', 'West Chelsea/Hudson Yards'],
    5: ['Garment District', 'Penn Station/Madison Sq West'],
    6: ['Midtown Center' ,'Midtown East', 'Midtown North', 'Times Sq/Theatre District'],
    7: ['Flatiron', 'Midtown South'],
    8: ['Gramercy', 'Kips Bay', 'Murray Hill', 'Stuy Town/Peter Cooper Village', 'Sutton Place/Turtle Bay North',
        'UN/Turtle Bay South', 'Union Sq'],
    9: ['Battery Park', 'Battery Park City', 'China Town', 'Greenwich Village North', 'Greenwich Village South',
        'Hudson Sq', 'Little Italy/NoLiTa', 'Meatpacking/West Village West', 'Seaport', 'SoHo', 'TriBeCa/Civic Center',
        'West Village', 'World Trade Center'],
    10: ['Alphabet City', 'East Village', 'Financial District North', 'Financial District South',
         'Lower East Side', 'Two Bridges/Seward Park']
}

not_manhattan=["Governor's Island/Ellis Island/Liberty Island",
               "Highbridge Park", "Marble Hill", "Randalls Island",
               "Roosevelt Island"]

In [6]:
zones_correspondence_ids={
    1: [],
    2: [],
    3: [],
    4: [],
    5: [],
    6: [],
    7: [],
    8: [],
    9: [],
    10: [],
}

for index, zone in taxi_zones[taxi_zones.Borough=='Manhattan']['Zone'].iteritems():
    for key in zones_correspondence.keys():
        if zone in zones_correspondence[key]:
            zones_correspondence_ids[key].append(index+1)

zones_correspondence_ids

{1: [41, 42, 43, 74, 75, 116, 127, 128, 243, 244],
 2: [24, 151, 152, 166, 238, 239],
 3: [140, 141, 236, 237, 262, 263],
 4: [48, 50, 68, 142, 143, 246],
 5: [100, 186],
 6: [161, 162, 163, 230],
 7: [90, 164],
 8: [107, 137, 170, 224, 229, 233, 234],
 9: [12, 13, 113, 114, 125, 144, 158, 209, 211, 231, 249, 261],
 10: [4, 79, 87, 88, 148, 232]}

In [7]:
zones_correspondence_ids_1to1={}

for (key, value) in zones_correspondence_ids.items():
    for ID in value:
        zones_correspondence_ids_1to1[ID]=key

In [7]:
taxi_data_jan=pd.read_csv('yellow_tripdata_2019-01.csv')
taxi_data_feb=pd.read_csv('yellow_tripdata_2019-02.csv')

taxi_data_jan.drop(columns=['congestion_surcharge', 'VendorID', 'store_and_fwd_flag', 
                            'extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge', 'RatecodeID'], inplace=True)
taxi_data_feb.drop(columns=['congestion_surcharge', 'VendorID', 'store_and_fwd_flag', 
                            'extra', 'mta_tax', 'tolls_amount', 'improvement_surcharge', 'RatecodeID'], inplace=True)


In [105]:
print(taxi_data_jan.isna().sum().sum())
print(taxi_data_feb.isna().sum().sum())
print(taxi_data_jan.info())

0
0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7667792 entries, 0 to 7667791
Data columns (total 11 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   tpep_pickup_datetime   object 
 1   tpep_dropoff_datetime  object 
 2   passenger_count        int64  
 3   trip_distance          float64
 4   RatecodeID             int64  
 5   PULocationID           int64  
 6   DOLocationID           int64  
 7   payment_type           int64  
 8   fare_amount            float64
 9   tip_amount             float64
 10  total_amount           float64
dtypes: float64(4), int64(5), object(2)
memory usage: 643.5+ MB
None


In [8]:
zones_to_keep=[]
for value in zones_correspondence_ids.values():
    zones_to_keep+=value

taxi_data_jan_manh=taxi_data_jan[(taxi_data_jan.PULocationID.isin(zones_to_keep)) & (taxi_data_jan.DOLocationID.isin(zones_to_keep))]
taxi_data_feb_manh=taxi_data_feb[(taxi_data_feb.PULocationID.isin(zones_to_keep)) & (taxi_data_feb.DOLocationID.isin(zones_to_keep))]

del taxi_data_jan
del taxi_data_feb

taxi_data_jan_manh['tpep_pickup_datetime']=pd.to_datetime(taxi_data_jan_manh['tpep_pickup_datetime'])
taxi_data_jan_manh['tpep_dropoff_datetime']=pd.to_datetime(taxi_data_jan_manh['tpep_dropoff_datetime'])
taxi_data_feb_manh['tpep_pickup_datetime']=pd.to_datetime(taxi_data_feb_manh['tpep_pickup_datetime'])
taxi_data_feb_manh['tpep_dropoff_datetime']=pd.to_datetime(taxi_data_feb_manh['tpep_dropoff_datetime'])

In [11]:
PUZone_jan=[]
DOZone_jan=[]
PUZone_feb=[]
DOZone_feb=[]

for zone in taxi_data_jan_manh['PULocationID']:
    PUZone_jan.append(zones_correspondence_ids_1to1[zone])

for zone in taxi_data_jan_manh['DOLocationID']:
    DOZone_jan.append(zones_correspondence_ids_1to1[zone])

for zone in taxi_data_feb_manh['PULocationID']:
    PUZone_feb.append(zones_correspondence_ids_1to1[zone])

for zone in taxi_data_feb_manh['DOLocationID']:
    DOZone_feb.append(zones_correspondence_ids_1to1[zone])

taxi_data_jan_manh['PULocationZone']=PUZone_jan
taxi_data_jan_manh['DOLocationZone']=DOZone_jan
taxi_data_feb_manh['PULocationZone']=PUZone_feb
taxi_data_feb_manh['DOLocationZone']=DOZone_feb

del PUZone_jan, DOZone_jan, PUZone_feb, DOZone_feb

In [12]:
taxi_data_feb_manh.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,fare_amount,tip_amount,total_amount,PULocationZone,DOLocationZone
0,2019-02-01 00:59:04,2019-02-01 01:07:27,1,2.1,48,234,1,9.0,2.0,12.3,4,8
4,2019-02-01 00:25:30,2019-02-01 00:28:14,1,0.8,140,263,2,5.0,0.0,6.3,3,3
5,2019-02-01 00:38:02,2019-02-01 00:40:57,1,0.8,229,141,2,4.5,0.0,5.8,8,3
6,2019-02-01 00:06:49,2019-02-01 00:10:34,1,0.9,75,41,2,5.0,0.0,6.3,1,1
7,2019-02-01 00:04:04,2019-02-01 00:24:27,1,2.8,246,229,2,14.0,0.0,15.3,4,8


In [13]:
taxi_data_jan_manh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6467338 entries, 0 to 7667787
Data columns (total 12 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   tpep_pickup_datetime   datetime64[ns]
 1   tpep_dropoff_datetime  datetime64[ns]
 2   passenger_count        int64         
 3   trip_distance          float64       
 4   PULocationID           int64         
 5   DOLocationID           int64         
 6   payment_type           int64         
 7   fare_amount            float64       
 8   tip_amount             float64       
 9   total_amount           float64       
 10  PULocationZone         int64         
 11  DOLocationZone         int64         
dtypes: datetime64[ns](2), float64(4), int64(6)
memory usage: 641.4 MB


In [8]:
def create_adjacency(df: pd.DataFrame, zones_correspondence_ids: dict) -> np.ndarray:
    mat=np.zeros((len(zones_correspondence_ids.keys()), len(zones_correspondence_ids.keys())))
    for i in range(df.shape[0]):
        PUzone=df.iloc[i]['PULocationZone']-1
        DOzone=df.iloc[i]['DOLocationZone']-1
        mat[PUzone, DOzone]+=1
        mat[DOzone, PUzone]+=1
    return mat


In [55]:
date=pd.date_range(start='2019-01-01', end='2019-02-01', freq='H')[0]
daily_df=taxi_data_jan_manh[(taxi_data_jan_manh.tpep_pickup_datetime>date) & (taxi_data_jan_manh.tpep_pickup_datetime<date+timedelta(hours=1))]
print(create_adjacency(daily_df, zones_correspondence_ids))


[[ 402.  220.  277.  126.   15.  110.   23.  110.   58.   45.]
 [ 220.  676.  323.  461.    9.   89.   25.  121.   97.   41.]
 [ 277.  323. 1544.  296.   36.  289.   71.  580.  209.  139.]
 [ 126.  461.  296. 1000.  105.  193.  167.  383.  423.  197.]
 [  15.    9.   36.  105.   28.   45.   46.  100.   62.   49.]
 [ 110.   89.  289.  193.   45.  286.   84.  347.  124.   91.]
 [  23.   25.   71.  167.   46.   84.   60.  264.  191.  149.]
 [ 110.  121.  580.  383.  100.  347.  264.  908.  452.  459.]
 [  58.   97.  209.  423.   62.  124.  191.  452.  864.  458.]
 [  45.   41.  139.  197.   49.   91.  149.  459.  458.  486.]]


In [57]:
taxi_data_jan_manh_aggregate=pd.DataFrame()
taxi_data_jan_manh_aggregate_Y=[]

for date in tqdm(pd.date_range(start='2019-01-01', end='2019-01-31 23:00:00', freq='H'), desc='Progress'):
    daily_df=taxi_data_jan_manh[(taxi_data_jan_manh.tpep_pickup_datetime>date) & (taxi_data_jan_manh.tpep_pickup_datetime<=date+timedelta(hours=1))]
    
    # Deprecated
    # taxi_data_jan_manh_aggregate.append({'tpep_pickup_datetime': date,
    #                                     'tpep_dropoff_datetime': date+timedelta(hours=1),
    #                                     'passenger_count': daily_df['passenger_count'].mean(),
    #                                     'trip_distance': daily_df['trip_distance'].mean(),
    #                                     'adjacency': create_adjacency(daily_df, zones_correspondence_ids)
    #                                     }, ignore_index=True)
    
    taxi_data_jan_manh_aggregate=pd.concat([taxi_data_jan_manh_aggregate, 
                                            pd.DataFrame({'hour_interval_lower': date,
                                                        'hour_interval_upper': date+timedelta(hours=1),
                                                        'passenger_count': daily_df['passenger_count'].mean(),
                                                        'trip_distance': daily_df['trip_distance'].mean()
                                            }, index=[0])], ignore_index=True)

    taxi_data_jan_manh_aggregate_Y.append(create_adjacency(daily_df, zones_correspondence_ids))

with open('taxi_data_jan_Y.pkl', 'wb') as f:
    pickle.dump(taxi_data_jan_manh_aggregate_Y, f)

Progress: 100%|██████████| 744/744 [29:17<00:00,  2.36s/it]


In [46]:
pd.date_range(start='2019-01-01', end='2019-01-31 23:00:00', freq='H')

DatetimeIndex(['2019-01-01 00:00:00', '2019-01-01 01:00:00',
               '2019-01-01 02:00:00', '2019-01-01 03:00:00',
               '2019-01-01 04:00:00', '2019-01-01 05:00:00',
               '2019-01-01 06:00:00', '2019-01-01 07:00:00',
               '2019-01-01 08:00:00', '2019-01-01 09:00:00',
               ...
               '2019-01-31 14:00:00', '2019-01-31 15:00:00',
               '2019-01-31 16:00:00', '2019-01-31 17:00:00',
               '2019-01-31 18:00:00', '2019-01-31 19:00:00',
               '2019-01-31 20:00:00', '2019-01-31 21:00:00',
               '2019-01-31 22:00:00', '2019-01-31 23:00:00'],
              dtype='datetime64[ns]', length=744, freq='H')

In [66]:
print(taxi_data_jan_manh_aggregate.describe())
print(taxi_data_jan_manh_aggregate.tail())

       passenger_count  trip_distance
count       744.000000     744.000000
mean          1.574364       1.840567
std           0.066214       0.203479
min           1.395867       1.539820
25%           1.523699       1.673708
50%           1.567714       1.758784
75%           1.626490       2.034582
max           1.758446       2.458458
    hour_interval_lower hour_interval_upper  passenger_count  trip_distance
739 2019-01-31 19:00:00 2019-01-31 20:00:00         1.543949       1.687769
740 2019-01-31 20:00:00 2019-01-31 21:00:00         1.556198       1.869081
741 2019-01-31 21:00:00 2019-01-31 22:00:00         1.588293       2.022072
742 2019-01-31 22:00:00 2019-01-31 23:00:00         1.589095       2.079742
743 2019-01-31 23:00:00 2019-02-01 00:00:00         1.602095       2.074673


In [75]:
print(taxi_data_jan_manh_aggregate_Y[-1])

[[122. 102. 116. 137.  20.  96.  35.  60.  42.  27.]
 [102. 336.  89. 210.  31. 111.  21.  58.  47.  18.]
 [116.  89. 570. 210. 113. 229. 105. 266. 111.  74.]
 [137. 210. 210. 648. 149. 391. 122. 285. 289. 111.]
 [ 20.  31. 113. 149.  44. 184.  48. 228. 129.  64.]
 [ 96. 111. 229. 391. 184. 370. 125. 367. 198. 129.]
 [ 35.  21. 105. 122.  48. 125.  70. 238. 187. 105.]
 [ 60.  58. 266. 285. 228. 367. 238. 684. 360. 280.]
 [ 42.  47. 111. 289. 129. 198. 187. 360. 780. 376.]
 [ 27.  18.  74. 111.  64. 129. 105. 280. 376. 366.]]


In [89]:
taxi_data_feb_manh_aggregate=pd.DataFrame()
taxi_data_feb_manh_aggregate_Y=[]

for date in tqdm(pd.date_range(start='2019-02-01', end='2019-02-28 23:00:00', freq='H'), desc='Progress'):
    daily_df=taxi_data_feb_manh[(taxi_data_feb_manh.tpep_pickup_datetime>date) & (taxi_data_feb_manh.tpep_pickup_datetime<=date+timedelta(hours=1))]
    
    # Deprecated
    # taxi_data_feb_manh_aggregate.append({'tpep_pickup_datetime': date,
    #                                     'tpep_dropoff_datetime': date+timedelta(hours=1),
    #                                     'passenger_count': daily_df['passenger_count'].mean(),
    #                                     'trip_distance': daily_df['trip_distance'].mean(),
    #                                     'adjacency': create_adjacency(daily_df, zones_correspondence_ids)
    #                                     }, ignore_index=True)
    
    taxi_data_feb_manh_aggregate=pd.concat([taxi_data_feb_manh_aggregate, 
                                            pd.DataFrame({'hour_interval_lower': date,
                                                        'hour_interval_upper': date+timedelta(hours=1),
                                                        'passenger_count': daily_df['passenger_count'].mean(),
                                                        'trip_distance': daily_df['trip_distance'].mean()
                                            }, index=[0])], ignore_index=True)

    taxi_data_feb_manh_aggregate_Y.append(create_adjacency(daily_df, zones_correspondence_ids))

with open('taxi_data_feb_Y.pkl', 'wb') as f:
    pickle.dump(taxi_data_feb_manh_aggregate_Y, f)

Progress: 100%|██████████| 672/672 [28:51<00:00,  2.58s/it]


In [90]:
with open('taxi_data_jan_tripdata.pkl', 'wb') as f:
    pickle.dump(taxi_data_jan_manh_aggregate, f)

with open('taxi_data_feb_tripdata.pkl', 'wb') as f:
    pickle.dump(taxi_data_feb_manh_aggregate, f)

## Join trips data with meteorological data

Firstly,  we have the aggregated trip data by hour:

In [10]:
with open('taxi_data_jan_tripdata.pkl', 'rb') as f:
    taxi_data_jan = pickle.load(f)

with open('taxi_data_feb_tripdata.pkl', 'rb') as f:
    taxi_data_feb = pickle.load(f)

print(taxi_data_jan.info())
print(taxi_data_feb.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 744 entries, 0 to 743
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   hour_interval_lower  744 non-null    datetime64[ns]
 1   hour_interval_upper  744 non-null    datetime64[ns]
 2   passenger_count      744 non-null    float64       
 3   trip_distance        744 non-null    float64       
dtypes: datetime64[ns](2), float64(2)
memory usage: 23.4 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   hour_interval_lower  672 non-null    datetime64[ns]
 1   hour_interval_upper  672 non-null    datetime64[ns]
 2   passenger_count      672 non-null    float64       
 3   trip_distance        672 non-null    float64       
dtypes: datetime64[ns](2), float64(2)
memor

Secondly, the meteorological data by day:

In [15]:
print(weather_data_jan.shape)
print(weather_data_feb.shape)
print(weather_data_jan.info())
# print(weather_data_feb.info())

(31, 17)
(28, 17)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Day            31 non-null     int64  
 1   Temp.Max       31 non-null     int64  
 2   Temp.Avg       31 non-null     float64
 3   Temp.Min       31 non-null     int64  
 4   DewPoint.Max   31 non-null     int64  
 5   DewPoint.Avg   31 non-null     float64
 6   DewPoint.Min   31 non-null     int64  
 7   Humidity.Max   31 non-null     int64  
 8   Humidity.Avg   31 non-null     float64
 9   Humidity.Min   31 non-null     int64  
 10  WindSpeed.Max  31 non-null     int64  
 11  WindSpeed.Avg  31 non-null     float64
 12  WindSpeed.Min  31 non-null     int64  
 13  Pressure.Max   31 non-null     float64
 14  Pressure.Avg   31 non-null     float64
 15  Pressure.Min   31 non-null     float64
 16  Precipitation  31 non-null     float64
dtypes: float64(8), int64(9)
memory usage: 

In [22]:
taxi_data_jan['hour_interval_lower'][0].day

pandas._libs.tslibs.timestamps.Timestamp

In [ ]:
taxi_data_jan_joined=pd.DataFrame()

for index, row in taxi_data_jan.iterrows():
    pass

# Modelling